In [1]:
import doctest

In [84]:
with open('input.txt', 'r') as f:
    raw_lines = f.readlines()

positions = [[int(c) for c in line.replace('\n', '').split(',')] for line in raw_lines]

# Part 1

In [85]:
def get_sorted_by_areas(positions):
    dmap = {}
    for (i, (x1, y1)) in enumerate(positions):
        for (j_offset, (x2, y2)) in enumerate(positions[i + 1:]):
            j = i + 1 + j_offset
            dmap[(i, j)] = abs(x1 - x2 + 1) * abs(y1 - y2 + 1)

    return sorted(dmap.keys(), key=lambda ij: dmap[ij])

In [ ]:
s = get_sorted_by_areas(positions)
part1_pair = s[-1]
i, j = part1_pair
len(s), positions[i], positions[j], (positions[i][0] - positions[j][0] + 1) * (positions[i][1] - positions[j][1] + 1)

# Part 2

In [81]:
def get_coord_lists_for_rect(pair, positions):
    i, j = pair
    p1, p2 = positions[i], positions[j]
    xs = [p1[0], p2[0], p2[0], p1[0], p1[0]]
    ys = [p1[1], p1[1], p2[1], p2[1], p1[1]]
    return xs, ys

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x_coords = np.array([p[0] for p in positions])
y_coords = np.array([p[1] for p in positions])

xs, ys = get_coord_lists_for_rect(part1_pair, positions)

gray_index = 248

plt.plot(x_coords, y_coords, marker='.', linestyle='-', color='red')
plt.plot([positions[0][0]], [positions[0][1]], marker='.', linestyle='-', color='blue')
plt.plot([positions[gray_index][0]], [positions[gray_index][1]], marker='.', linestyle='-', color='gray')
plt.plot(xs, ys, marker='.', linestyle='-', color='purple')
plt.grid(True)
plt.show()

In [ ]:
# By eye, we can see the rectangle cannot cross the middle section
print(positions[gray_index - 1], positions[gray_index], positions[gray_index + 1], positions[gray_index + 2])

# let's just say the answer cannot cross this "ybar" range
y_bar_min = positions[gray_index + 1][1]
y_bar_max = positions[gray_index][1]
print(y_bar_min, y_bar_max)

In [5]:
# If there are any other red tiles in the rectangle, it is disqualified.
# Red tiles on the edge are fine because they are connected by green.
def are_no_red_tiles_in_rect(x1, y1, x2, y2, positions, i, j):
    return not any(min(x1, x2) < x3 < max(x1, x2) and
                        min(y1, y2) < y3 < max(y1, y2)
                        for k, (x3, y3) in enumerate(positions) if k not in (i, j))

In [14]:
def get_area(i, j, positions):
    return abs(positions[i][0] - positions[j][0] + 1) * abs(positions[i][1] - positions[j][1] + 1)

In [71]:

def get_sorted_by_areas_2(positions):
    dmap = {}
    for (i, (x1, y1)) in enumerate(positions):
        for (j_offset, (x2, y2)) in enumerate(positions[i + 1:]):
            j = i + 1 + j_offset
            if are_no_red_tiles_in_rect(x1, y1, x2, y2, positions, i, j) and \
                    ((y1 <= y_bar_min and y2 <= y_bar_min) or (y1 >= y_bar_max and y2 >= y_bar_max)):
                dmap[(i, j)] = abs(x1 - x2 + 1) * abs(y1 - y2 + 1)

    return sorted(dmap.keys(), key=lambda ij: dmap[ij])

In [72]:
sorted_2 = get_sorted_by_areas_2(positions)

In [ ]:
corners = sorted_2[-1]
xs, ys = get_coord_lists_for_rect(corners, positions)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

for ax in (axes[0, 0], axes[1, 0], axes[1, 1]):
    ax.plot(x_coords, y_coords, marker='.', linestyle='-', color='red')
    ax.plot(xs, ys, marker='.', linestyle='-', color='purple')
    ax.grid(True)

axes[0, 0].set_title(f'points {corners}, area {get_area(corners[0], corners[1], positions) / 1e6}')

axes[1, 0].set_ylim(30000,70000)
axes[1, 0].set_xlim(0, 10000)

axes[1, 1].set_ylim(30000,70000)
axes[1, 1].set_xlim(85000, 100000)

plt.show()

In [ ]:
get_area(corners[0], corners[1], positions)

# Part 2 - scratch

In [6]:
def is_inside(point, positions):
    # count how many intervals we'd cross if we go all the way to the left.
    # if it's an odd number, we're inside the loop.
    # this doesn't need to work with integer coordinates
    """
    >>> positions = [[7, 1], [11, 1], [11, 7], [9, 7], [9, 5], [2, 5], [2, 3], [7, 3]]
    >>> is_inside((4.1, 4), positions)
    True
    >>> is_inside((20, 4.1), positions)
    False
    >>> is_inside((10, 3.1), positions)
    True
    """
    x, y = point
    wrapped_pairs = zip(positions, positions[1:] + [positions[0]])

    if y != int(y):
        # If y is off the lines, go to the left edge to count crossings
        crossings = sum(
            1
            for (x1, y1), (x2, y2) in wrapped_pairs
            if x > x1 and x > x2 and y >= min(y1, y2) and y <= max(y1, y2))
    elif x != int(x):
        # if x is off the lines, go to the top edge to count crossings
        crossings = sum(
            1
            for (x1, y1), (x2, y2) in wrapped_pairs
            if y > y1 and y > y2 and x >= min(x1, x2) and x <= max(x1, x2))
    else:
        raise Exception('cannot tell if inside')

    return crossings % 2 == 1

doctest.testmod()

TestResults(failed=0, attempted=4)

In [7]:
import math

def get_fractional_point_on_line(x1, y1, x2, y2):
    """
    is_inside needs a fractional coordinate,
    so just interpolate a little bit.

    >>> get_fractional_point_on_line(1, 2, 6, 2)
    (1.1, 2.0)
    >>> get_fractional_point_on_line(3, 1, 3, -1)
    (3, -0.75)
    """
    length = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    f = 1 / (2 * length)
    if x1 != x2:
        m = (y2 - y1) / (x2 - x1)
        x = min(x1, x2) + f
        y = min(y1, y2) + f * m
    else:
        x = x1
        y = min(y1, y2) + f
    return x, y

doctest.testmod()

TestResults(failed=0, attempted=6)

In [ ]:
def get_sorted_2_with_inside_check(positions):
    dmap = {}
    for (i, (x1, y1)) in enumerate(positions):
        for (j_offset, (x2, y2)) in enumerate(positions[i + 1:]):
            j = i + 1 + j_offset
            if are_no_red_tiles_in_rect(x1, y1, x2, y2, positions, i, j) and \
                    is_inside(get_fractional_point_on_line(x1, y1, x2, y2), positions):
                dmap[(i, j)] = abs(x1 - x2 + 1) * abs(y1 - y2 + 1)

    return sorted(dmap.keys(), key=lambda ij: dmap[ij])

In [ ]:
# The answer surely needs to include one of those two points that jut in to the ellipse.
# These are points 248 and 249.

for i, pair in enumerate(zip(positions, positions[1:] + [positions[0]])):
    p1, p2 = pair
    l2 = (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2
    if l2 > 10000**2:
        break
print(i)

247


In [22]:
must_haves = [248, 249]

restricted = [(i, j) for i, j in sorted_2 if i in must_haves or j in must_haves]

In [192]:
restricted[-26]

(249, 270)

In [ ]:
get_area(249, 269, positions)